In [13]:
%load_ext Cython

In [57]:
a = np.array([3,4,3,4], dtype=int)

4

In [49]:
%%cython -c=-std=c++17 -lseal
# distutils: language = c++

# import both numpy and the Cython declarations for numpy
import numpy as np
cimport numpy as np
cimport cython

# Import from Cython libs required C/C++ types for the Afhel API
from libcpp.vector cimport vector
from libcpp.string cimport string
from libcpp cimport bool
from libc.stdint cimport int64_t
from libc.stdint cimport uint64_t

# Import our own wrapper for iostream classes, used for I/O ops
from iostream cimport istream, ostream, ifstream, ofstream   

from Afhel cimport Plaintext
from Afhel cimport Ciphertext
from Afhel cimport Afseal


from PyCtxt cimport PyCtxt
from PyPtxt cimport PyPtxt

ctypedef fused plaintext_t:
    cython.double
    int64_t
    vector[int64_t]
    vector[double]

In [81]:
he = Pyfhel()

In [82]:
he.ContextGen(2)

In [83]:
he.KeyGen()

In [84]:
c1 = he.encryptVal(3)

In [86]:
type(c1)

PyCtxt.PyCtxt

In [80]:
%%cython -c=-std=c++17 -lseal -lafhel
# distutils: language = c++

# import both numpy and the Cython declarations for numpy
import numpy as np
cimport numpy as np
cimport cython

# Dereferencing pointers in Cython in a secure way
from cython.operator cimport dereference as deref

# Import from Cython libs required C/C++ types for the Afhel API
from libcpp.vector cimport vector
from libcpp.string cimport string
from libcpp cimport bool
from libc.stdint cimport int64_t
from libc.stdint cimport uint64_t

# Import our own wrapper for iostream classes, used for I/O ops
from iostream cimport istream, ostream, ifstream, ofstream   

from Afhel cimport Plaintext
from Afhel cimport Ciphertext
from Afhel cimport Afseal

ctypedef fused double_int64:
    cython.double
    int64_t
    
ctypedef fused vec_double_int64:
    vector[cython.double]
    vector[int64_t]

# Import the Plaintext and Cyphertext classes for Python
from PyPtxt cimport PyPtxt
from PyCtxt cimport PyCtxt


cdef class Pyfhel:
    cdef Afseal* afseal           # The C++ methods are accessed via a pointer
    
    # INIT & DESTRUCT
    def __cinit__(self):
        self.afseal = new Afseal()
    def __dealloc__(self):
        if self.afseal != NULL:
            del self.afseal

    
    # ----------------------------- CRYPTOGRAPHY ------------------------------
    # CONTEXT & KEY GENERATION
    cpdef void ContextGen(self, long p, long m=2048, bool flagBatching=False,
                  long base=2, long sec=128, int intDigits=64, int fracDigits = 32):
        self.afseal.ContextGen(p, m, flagBatching, base, sec,intDigits, fracDigits)
        
    cpdef void KeyGen(self):
        self.afseal.KeyGen()
    
    cpdef encryptVal(self, double_int64 val, PyCtxt ctxt=None):
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(val, deref(ctxt._ptr_ctxt))
        return ctxt
    
    cpdef encryptBatch(self, np.ndarray[int64_t, ndim=1, mode="c"] vec, PyCtxt ctxt=None): 
        cdef vector[int64_t]* ptr_vec
        ptr_vec.assign(&vec[0], &vec[0] + vec.shape[0])
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(deref(ptr_vec), deref(ctxt._ptr_ctxt)) 
        return ctxt  
        
    # ENCRYPTION encrypt a PyPtxt object into a PyCtxt object
    cpdef encrypt(self, PyPtxt ptxt, PyCtxt ctxt=None):
        if (ptxt._ptr_ptxt == NULL):
            raise TypeError("Plaintext is empty")
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(deref(ptxt._ptr_ptxt), deref(ctxt._ptr_ctxt)) 
        return ctxt


In [ ]:

    # DECRYPT a PyCtxt object into a List of values
    def decrypt(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel decrypt error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))
        retList = []
        cdef vector[long] retVect
        ids = ctxt.getIDs()
        lens = ctxt.getLen()
        for i,l in zip(ids,lens):           # For each key in PyCtxt
            retPtxt = []
            retV = self.afseal.decrypt(i)    # Decrypt its Afhel Ctxt
            for k in range(l):
                retPtxt.append(retV[k])     # Append values 1 by 1 to final list
            retList.append(retPtxt)
        return retList                      # Return only the non-filler values


    # DUPLICATE a PyCtxt with all its parameters, useful to keep originals in ops
    def duplicate(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel set error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))
        ids = ctxt.getIDs()                 # Use same Pyfhel class and same #IDs
        new_ctxt = PyCtxt(ctxt.getPyfhel(), ctxt.getLen())
        for i in ids:                       # Loop over all IDs and append them
            new_ctxt.appendID(self.afseal.set(i))
        return new_ctxt                     # Return duplicated PyCtxt


    # ----------------------------- OPERATIONS --------------------------------
    # ADD two PyCtxt objects for each ID in both
    def add(self, ctxt1, ctxt2, neg=False):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel addCtxt error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel addCtxt error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))

        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::add to + each pair of Ctxts by IDs
            self.afseal.add(ids1[i],ids2[i], neg)


    # MULTiply two PyCtxt objects for each ID in both
    def mult(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::mult to * each pair of Ctxts by IDs
            self.afseal.mult(ids1[i],ids2[i])


    # MULTIPLY 3 PyCtxt objects for each ID in both
    def mult3(self, ctxt1, ctxt2, ctxt3):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        if not isinstance(ctxt3, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt3 must be of type PyCtxt instead of type " + str(type(ctxt3)))    
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        ids3 = ctxt3.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        if n_ids != len(ids3):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::mult to * each pair of Ctxts by IDs
            self.afseal.mult3(ids1[i],ids2[i], ids3[i])




    # SCALAR PRODuct between two PyCtxt objects for each ID in both
    def scalarProd(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel scalarProd error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel scalarProd error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::scalarProd to compute each pair of Ctxts by IDs
            self.afseal.scalarProd(ids1[i],ids2[i], 0)
            

    # SQUARE each cyphertext inside PyCtxt ctxt for each ID in it
    def square(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel square error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.square(ids[i])




    # CUMSUM Cumulative sum over all the values in the cyphertext
    def cumSum(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel cube error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.cumSum(ids[i])




    # CUBE each cyphertext inside PyCtxt ctxt for each ID in it
    def cube(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel cube error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.cube(ids[i])


    # NEGATE each cyphertext inside PyCtxt ctxt for each ID in it
    def negate(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel negate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.negate(ids[i])


    # COMPARE two PyCtxt objects for each ID in both
    def equalsTo(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel equalsTo error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel equalsTo error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        comparison = []
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::equalsTo to * each pair of Ctxts by IDs
            comparison.append(self.afseal.mult(ids1[i],ids2[i]))
        return comparison


    # ROTATE each cyphertext inside PyCtxt ctxt for each ID in it
    def rotate(self, ctxt, c):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel rotate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.rotate(ids[i], c)


    # SHIFT each cyphertext inside PyCtxt ctxt for each ID in it
    def shift(self, ctxt, c):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel rotate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.shift(ids[i], c)



    # ----------------------------------- I/O ---------------------------------

    # SAVE ENVIRONMENT
    # Saves the environment into a .aenv file
    def saveEnv(self, fileName):
        if not isinstance(fileName, str):
            raise TypeError("Pyfhel saveEnv error: fileName must be of type str instead of type " + str(type(fileName)))
        self.afseal.saveEnv(fileName)
        return 


    # RESTORE ENVIRONMENT
    # Restores the environment from a .aenv file
    def restoreEnv(self, fileName):
        if not isinstance(fileName, str):
            raise TypeError("Pyfhel saveEnv error: fileName must be of type str instead of type " + str(type(fileName)))
        self.afseal.restoreEnv(fileName)
        self.modulus = long(pow(self.afseal.getP(), self.afseal.getR()))
        return

    #--------------------------------- AUXILIARY ------------------------------
    def numSlots(self):
        return self.afseal.numSlots()
    def getModulus(self):
        return self.modulus
    def delete(self, ctxt):
        for i in ctxt.getIDs():
            self.afseal.erase(i)



EXAMPLE ON USAGE OF STD STREAMS FROM C++

In [16]:
%%cython
# distutils: language = c++
from iostream cimport *    

def test_ofstreamss(bytes s):
    cdef ofstream* outputter
    # use try ... finally to ensure destructor is called
    outputter = new ofstream("output.txt")
    try:
        outputter.write(s,len(s))
    finally:
        del outputter
        
def test_ifstreamss(int n):
    cdef ifstream* inputer
    inputer = new ifstream("output.txt")
    s = b'aaaaaa'
    try:
        inputer.read(s,n)
    finally:
        del inputer
    return s

In [17]:
test_ofstreamss(b"This is a test\n")

In [18]:
test_ifstreamss(6)

b'This i'